In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

In [24]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, TensorBoard
import h5py
from keras.models import load_model

In [5]:
train_data_path = '/home/ec2-user/Capstone_Project/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask/train/'
test_data_path = '/home/ec2-user/Capstone_Project/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask/test/'

## Visualize Masks

In [ ]:
files = []
folder = 'mask/train/'
path = []
for filename in os.listdir('mask/train/'):
    files.append(filename)
    path.append('{}{}'.format(folder,filename))

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.ravel()
for i in range(len(path)):
    img = load_img(path=path[i],grayscale=True, target_size=(150,150), interpolation='nearest')
    axs[i].imshow(img)
    axs[i].set_title(files[i])

In [ ]:
files = []
folder = 'mask/test/'
path = []
for filename in os.listdir('mask/test/'):
    files.append(filename)
    path.append('{}{}'.format(folder,filename))

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.ravel()
for i in range(len(path)):
    img = load_img(path=path[i],grayscale=True, target_size=(150,150), interpolation='nearest')
    axs[i].imshow(img)
    axs[i].set_title(files[i])

In [ ]:
# Read data form training and test 

In [8]:
X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
                                             masks_path = train_mask_path, 
                                             crop_size = 33, 
                                             num_classes = 6, 
                                             samples = 1000, 
                                             balance = True)

In [9]:
X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
                                           masks_path = test_mask_path, 
                                           crop_size = 33, 
                                           num_classes = 6, 
                                           samples = 1000, 
                                           balance = True)

In [10]:
# creat generato

In [11]:
print(type(X_train), type(Y_train))
print(X_train.shape, Y_train.shape, num_bands)
print(X_test.shape,  Y_test.shape,  num_bands)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(6000, 33, 33, 16) (6000, 6) 16
(6000, 33, 33, 16) (6000, 6) 16


In [12]:
X_train = np.array([X_train[i] for i in range(X_train.shape[0])])

In [13]:
X_test = np.array([X_test[i] for i in range(X_test.shape[0])])

In [14]:
Y_train = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [15]:
Y_test = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# Build First Model

In [31]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(33, 33, 16)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(6))
# model.add(Activation('softmax'))

model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(33, 33, 16),
                 kernel_regularizer=l2(0.01)))

model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(5, 5),
                 strides=(1, 1),
                 activation='relu',
                 kernel_regularizer=l2(0.01)))

model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2, 2)))
model.add(Flatten())
model.add(Dense(128,
                activation='relu',
                kernel_regularizer=l2(0.01)))
model.add(Dense(6,
                activation='softmax',
                kernel_regularizer=l2(0.01)))

In [37]:
filepath="Model_1_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
#callbacks_list = [checkpoint,tbCallBack]

In [38]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [39]:
model.fit(X_train, Y_train,
          epochs = 30,
          verbose = 1,
          batch_size= 50,
          validation_data=(X_test, Y_test),
          callbacks = [tbCallBack] )

Train on 6000 samples, validate on 6000 samples
Epoch 1/30
6000/6000 [==============================] - 2s 343us/step - loss: 0.3219 - acc: 0.9427 - val_loss: 1.0216 - val_acc: 0.7133
Epoch 2/30
6000/6000 [==============================] - 2s 280us/step - loss: 0.3423 - acc: 0.9402 - val_loss: 1.3237 - val_acc: 0.6668
Epoch 3/30
6000/6000 [==============================] - 2s 290us/step - loss: 0.3226 - acc: 0.9472 - val_loss: 1.4526 - val_acc: 0.6553
Epoch 4/30
6000/6000 [==============================] - 2s 290us/step - loss: 0.3070 - acc: 0.9508 - val_loss: 1.0184 - val_acc: 0.7008
Epoch 5/30
6000/6000 [==============================] - 2s 281us/step - loss: 0.3105 - acc: 0.9490 - val_loss: 0.9841 - val_acc: 0.7093
Epoch 6/30
6000/6000 [==============================] - 2s 280us/step - loss: 0.3146 - acc: 0.9480 - val_loss: 0.9617 - val_acc: 0.7155
Epoch 7/30
6000/6000 [==============================] - 2s 281us/step - loss: 0.3095 - acc: 0.9502 - val_loss: 1.1652 - val_acc: 0.6612


In [25]:
model = load_model('Model_1_best.h5')

In [26]:
# this make aure all the layers are trainable
for layer in model.layers:
    print(layer.trainable,layer.name)

True conv2d_1
True max_pooling2d_1
True conv2d_2
True max_pooling2d_2
True flatten_1
True dense_1
True dense_2


ModuleNotFoundError: No module named '__main__._conv'; '__main__' is not a package